## Simulation Configuration
Let's demonstrate how to efficiently simulate the Landscape Model using the stimuli from our SBS dataset.

### Dependencies and Data

In [2]:
import pandas as pd
import numpy as np
import json
from psifr import fr

# load collection of similarity matrices and convert to numpy
with open('data/similarities.json', 'r') as f:
    connections = json.load(f)
for key in connections.keys():
    connections[key] = np.array(connections[key])

# load recall data frame
data = pd.read_csv('data/psifr_sbs.csv')
events = fr.merge_free_recall(
    data, list_keys=['item_index', 'cycle', 'story_index', 
                     'story_name', 'time_test'])
events.head()

,subject,list,item,input,output,study,recall,repeat,intrusion,item_index,cycle,story_index,story_name,time_test
0,0,0,One fine day an old Maine man was fishing,1.0,NaN,True,False,0,False,0,0,0,Fisherman,1
1,0,0,on his favorite lake,2.0,2.0,True,True,0,False,1,0,0,Fisherman,1
2,0,0,and catching very little,3.0,NaN,True,False,0,False,2,0,0,Fisherman,1
3,0,0,"Finally, he gave up",4.0,3.0,True,True,0,False,3,1,0,Fisherman,1
4,0,0,and walked back along the shore to his fishing...,5.0,NaN,True,False,0,False,4,1,0,Fisherman,1


### Cycle Extraction
Let's just directly get a list of cycles containing indices of relevant units.

In [3]:
cycle_table = events.pivot_table(index=['story_name'], columns='input', values='cycle')
cycle_table

input,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0
story_name,,,,,,,,,,,,,,,,,,,,,
Beach,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,2.0,...,10.0,10.0,11.0,11.0,12.0,12.0,12.0,NaN,NaN,NaN
Cat,0.0,0.0,0.0,0.0,1.0,2.0,3.0,4.0,5.0,5.0,...,24.0,24.0,24.0,24.0,25.0,25.0,NaN,NaN,NaN,NaN
Fisherman,0.0,0.0,0.0,1.0,1.0,2.0,3.0,4.0,5.0,6.0,...,24.0,25.0,26.0,27.0,28.0,28.0,29.0,30.0,NaN,NaN
Flight,0.0,1.0,1.0,2.0,3.0,4.0,4.0,4.0,4.0,4.0,...,12.0,12.0,12.0,13.0,14.0,15.0,15.0,16.0,17.0,18.0
Fog,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Supermarket,0.0,0.0,1.0,1.0,2.0,2.0,2.0,3.0,4.0,4.0,...,17.0,18.0,18.0,19.0,19.0,19.0,NaN,NaN,NaN,NaN


In [4]:
experiences = {}

for story_name in connections.keys():
    v = cycle_table.loc[story_name].values
    
    next_experience = []
    current_cycle = 0
    experiences[story_name] = []

    for unit_index, cycle_index in enumerate(v):
        if current_cycle != cycle_index:
            experiences[story_name].append(next_experience)
            next_experience = [unit_index]
            current_cycle = cycle_index
        else:
            next_experience.append(unit_index)

### Recall Rates
We're going to be testing the correlation between connection strengths and unit recall rates for each story, across subjects.

In [5]:
recall_rates = events.loc[(events.time_test==2)].pivot_table(index='input', columns='story_name', values='recall')
recall_rates.head()

story_name,Beach,Cat,Fisherman,Flight,Fog,Supermarket
input,,,,,,
1.0,1.000000,0.545455,0.500000,1.000000,0.590909,1.000000
2.0,0.909091,0.681818,0.909091,0.909091,0.545455,0.954545
3.0,0.909091,0.772727,0.454545,1.000000,0.909091,0.909091
4.0,0.909091,0.727273,0.954545,0.954545,0.909091,0.636364
5.0,0.454545,0.454545,0.863636,0.772727,1.000000,0.590909


### Simulation

In [6]:
connection_strengths = {}
distance_ranks = {}

for story_name in connections.keys():
    connection_strengths[story_name] = []
    distance_ranks[story_name] = []

for story_name in connections.keys():
    model = LandscapeRevised(connections[story_name])
    model.connections[np.eye(model.unit_count, dtype='bool')] = 0
    
    # track connection strengths for each unit
    connection_strengths[story_name].append(np.sum(model.connections, axis=0))
    
    # track distance_rank score across units
    distance_rank = fr.distance_rank(
        events.loc[(events.story_name==story_name) & (events.time_test==1)], 
        'item_index', 1-model.connections).agg(['mean'])
    distance_ranks[story_name].append(distance_rank)


c:\programdata\miniconda3\lib\site-packages\psifr\measures.py:298: RuntimeWarning: Mean of empty slice
  {'subject': subject, 'rank': np.nanmean(ranks)}, index=[subject]
c:\programdata\miniconda3\lib\site-packages\psifr\measures.py:298: RuntimeWarning: Mean of empty slice
  {'subject': subject, 'rank': np.nanmean(ranks)}, index=[subject]


In [7]:
from scipy import stats

all_recall_rates = []
all_connection_strengths = []
for story_name in connections.keys():
    print(story_name)
    
    for i in range(len(connection_strengths[story_name])):
        story_recall_rates = recall_rates[story_name].values
        story_recall_rates = story_recall_rates[np.logical_not(np.isnan(story_recall_rates))]
        all_recall_rates = all_recall_rates + story_recall_rates.tolist()

        all_connection_strengths = all_connection_strengths + connection_strengths[story_name][i].tolist()
    
stats.pearsonr(all_connection_strengths, all_recall_rates)

Fisherman
Supermarket
Flight
Cat
Fog
Beach


(-0.2751566373180548, 2.513792465699382e-05)

In [8]:
connections

{'Fisherman': array([[ 1.00000036,  0.35791534,  0.32181364, ...,  0.23844315,
          0.3343226 ,  0.3050786 ],
        [ 0.35791534,  0.99999988,  0.14880559, ...,  0.11115991,
         -0.01238859,  0.03861837],
        [ 0.32181364,  0.14880559,  0.99999994, ...,  0.34335035,
          0.17616002,  0.11304599],
        ...,
        [ 0.23844315,  0.11115991,  0.34335035, ...,  1.        ,
          0.13280995,  0.1272001 ],
        [ 0.3343226 , -0.01238859,  0.17616002, ...,  0.13280995,
          1.00000024,  0.4298729 ],
        [ 0.3050786 ,  0.03861837,  0.11304599, ...,  0.1272001 ,
          0.4298729 ,  1.00000048]]),
 'Supermarket': array([[0.99999988, 0.32480338, 0.63267267, ..., 0.21498573, 0.1301775 ,
         0.18464422],
        [0.32480338, 1.00000083, 0.37761265, ..., 0.22474755, 0.03419457,
         0.33486927],
        [0.63267267, 0.37761265, 1.00000024, ..., 0.33856726, 0.15549192,
         0.30914748],
        ...,
        [0.21498573, 0.22474757, 0.33856726,

## Exploration

We need some testing code to help confirm whether the model actually works or not as we develop it. What is our metric? Developing our `Semantic_Effects` notebook would clarify this. 

In the 6/15 report, I specify a few analyses. One is model fits, another is clustering by representational similarity, another is a vague collection of semantic and temporal organizational analyses. The semantic analyses I had in mind were the lag-rank analyses supported by the psifr toolbox. The clustering analyses were also lag-rank analyses, I guess, but using representations simulated using the Landscape Model! And then the almost simpler analyses demoed in the Yeari et al (2016) and Cutler et al (2019) papers, where we measure the correlation between the connectivity between idea units (measured as the mean or sum of similarities between each idea unit and every other idea unit) against the recall rates of each idea unit.

Let's try to build a visualization that starts with the baseline connection weights but also tracks statistics across each update of the Landscape model. Since even the very first version of the model will have those initial weights, this gives me a clean way to scale up and do that all within the constrained objectives of this notebook.

This is doing what? Initialize the model with relevant connections - also presumably loading relevant data and figuring out how to encode individual cycles. Then at the end of each cycle simulation, I want to do the lag_rank analysis, getting a scalar representing the extent to which recall is clustered on the basis of my distance metric.

Do this for each story. Plot a line for each story representing the timecourse of each score as the representation evolves.

At the end of all simulations, plot for both initial and simulated connections the correlation between summed connectivity and unit recall rate.

## Notes

### How do we want to multiply activations and sigma?

It's multiplying each connection value in row i with each corresponding activation value j, then summing over column. Is that better?

>  The activation spread from a connected unit j to a target unit i equals the multiplication of the activation of j from the previous cycle (Ajc−1) with the connection strength between units i and j as computed in the previous cycle (Sijc−1)

Each index in activation corresponds to to the activation of j from the previous cycle. The connection strength between unit i and j as computed in the previous cycle is at index (i, j). So the right values are being accessed? Well, what is being summed over? Before we compute our sum, we are tracking for each connected unit j its activation (a[j]) times its connection with unit i (c[i, j]). The summation of this vector (collapsing over all connected units) reflects the activation spread to target unit i. So it's not 0, 22, 76, 162 that we want here, but 14, 38 62, 86.

So yeah, we just want the dot product.

In [44]:
connections = np.arange(16).reshape(4, 4)
activations = np.arange(4)

activations, connections

(array([0, 1, 2, 3]),
 array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]]))

In [45]:
connections @ activations

array([14, 38, 62, 86])

In [46]:
activations @ connections

array([56, 62, 68, 74])

Since our connections matrix is always symmetric, in practice we'd see the same result for either operation order.

### How Many Parameters Does This Model _Really_ Have?

Outside of CMR, it's sort of 5? We list:

- connections: array, initial connection strengths between text units  
- max_activity: maximum activation that units are allowed to have  
- min_activity: minimum activation that units are allowed to have  
- decay_rate: decay rate of unit activation from one cycle to next  
- memory_capacity: total activation possible in any given cycle  
- learning_rate: rate of connection weight changes across cycles  
- semantic_strength: relative contribution of initial semantic  
    connections to computation of overall connection strengths  
    
Connections are static within a DSM (we could individualize this in a different project). `min_activity` should technically always be 0 according to the specification in Yeari et al. THe rest really do depend on individual differences according to the model. 

If we decide to integrate the model into CMR, some parameters seem like they should correspond with some parameters in CMR. CMR has its own learning rate parameters, its own drift rate parameters, and its own scalaing of semantic association strengths. These of course have different consequences, and to use the same parameters would be to propose some shared representational substrate. An interesting project might be to explore the extent to which parameters "correlate" between subjects.

### Default Parameters

> The default values were chosen on the basis of theoretical grounds (e.g., working memory capacity is five times the maximal value of the text units; Kintsch, 1988), neutrality (e.g., λ is close to 1, the semantic strength coefficient equals 1), and computational constraints (e.g., δ is set to .1 to prevent text units from noncurrent cycles from reaching the maximal activation value).

> All simulations used the same default values set by the model in the following parameters: (1) semantic strength coefficient = 1, (2) maximal activation = 1, (3) working memory capacity = 5, (4) learning rate = .9, and (5) activation decay rate = .1.